## Training and Inference NN with Pytorch

A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: ```weight = weight - learning_rate * gradient```

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Define neural network

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [5]:
for param in net.parameters():
    print(param.shape)

torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [6]:
param.requires_grad

True

In [7]:
# Forward propagation
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0936,  0.0239,  0.0156, -0.0262,  0.0839,  0.0794, -0.0021,  0.0840,
          0.0389, -0.0636]], grad_fn=<AddmmBackward>)


In [8]:
# Zero the gradient buffers of all parameters and backprops with random gradients:
net.zero_grad()
out.backward(torch.randn(1, 10))

### Recap:

- ```torch.Tensor``` - A multi-dimensional array with support for autograd operations like backward(). Also holds the gradient w.r.t. the tensor.
- ```nn.Module``` - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
- ```nn.Parameter``` - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module.
- ```autograd.Function``` - Implements forward and backward definitions of an autograd operation. Every Tensor operation creates at least a single Function node that connects to functions that created a Tensor and encodes its histor

## Loss Function

In [30]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
print("#1", target.size())
target = target.view(1, -1)  # make it the same shape as output
print("#2", target.size())

criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

#1 torch.Size([10])
#2 torch.Size([1, 10])
tensor(0.5052, grad_fn=<MseLossBackward>)


In [31]:
print(loss.grad_fn)  # MSELoss

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

In [32]:
print(loss.grad_fn.next_functions[0][0])  # Linear

In [33]:
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [34]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0012,  0.0101, -0.0104,  0.0048,  0.0050, -0.0012])
